In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import rasterio

# Notebook to generate head obs from DEC wells

### Read in and merge DEC well data. Souce data from:
* NYDEC downloadable well database: https://www.dec.ny.gov/energy/1603.html
* NYDEC water well database, via NYS GIS Clearinghouse: https://gis.ny.gov/gisdata/inventories/details.cfm?DSID=1203



In [ ]:
dec_wells = gp.read_file('../source_data/Shapefiles/Wells/NYDEC_All_wells_clipped.shp')
water_wells = gp.read_file('../source_data/Shapefiles/Wells/NYDEC_water_wells_clipped.shp')

In [ ]:
#  read in basin boundary for plotting
bound = gp.read_file('../source_data/Shapefiles/Extents/Model_Extent_HUC12.shp')
bound = bound.to_crs(epsg=5070)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
bound.plot(ax=ax, edgecolor='k', facecolor='none')
dec_wells.plot(ax=ax)
plt.title('All DEC wells')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
bound.plot(ax=ax, edgecolor='k', facecolor='none')
water_wells.plot(ax=ax, c='red')
plt.title('DEC wells from water wells database')
plt.show()

## Combine the the two DEC well dataframes and work up into an obs file

In [ ]:
dec_wells['group'] = 'dec_all'
dec_wells['obsnme'] = dec_wells['WellNO'] #  make common obsname

In [ ]:
water_wells['group'] = 'dec_water_wells'
water_wells['obsnme'] = water_wells['DEC_Well_N'] #  make common obsname

In [ ]:
dec_wells_all = dec_wells.append(water_wells, ignore_index=True)
dec_wells_all

In [ ]:
dec_wells_all_working = dec_wells_all.copy()

#  remove all wells with null GW_Depth
dec_wells_all_working = dec_wells_all_working.loc[dec_wells_all_working.GW_Depth.notnull()]

#  assume that zeros are null? Maybe not always the case but hard to know the zeros are real
dec_wells_all_working = dec_wells_all_working.loc[dec_wells_all_working.GW_Depth.astype(float) != 0]

#  finally, drop the 55 duplicates in 'dec_water_wells' 
dec_wells_all_working = dec_wells_all_working.drop_duplicates(subset=['obsnme'], keep='first') 

Land surface elevations in the DEC wells table look pretty unreliable. Want to get reasonable land surface elevations. Extract pixel values from land surface elevation raster at the location of each well. This probably improves land surface elevation values, from which we can calculate better groundwater elevations from the supplied depth to water values... assuming those depth to water values are at all reliable.

In [ ]:
def raster_point_value(input_raster, gdf):
    """
    Extracts raster pixel value from the well/point locations listed in a geodataframe.

    Properties
    ------------------------------
    input_raster: str, path to categorical landuse raster file for analysis
    gdf: geopandas geodataframe, contains of geometry of wells/points

    """

    #  extract raster crs
    with rasterio.open(input_raster) as src:
        raster_crs = src.crs

    #  transform geodataframe to raster crs
    gdf_reproj = gdf.to_crs(raster_crs)

    #  get a list of coordinate tuples for each row in a geopandas dataframe that contains point
    x_coords = gdf_reproj['geometry'].x.to_list()
    y_coords = gdf_reproj['geometry'].y.to_list()
    coords = list(zip(x_coords, y_coords))

    #  extract raster cell values at well/point coordinates
    with rasterio.open(input_raster) as src:
        point_values = list(np.concatenate([v for v in src.sample(coords)], axis=0))

    return point_values

In [ ]:
input_raster = '../source_data/Shapefiles/top_50m_from_lidar.tif'

#  use function above to get list of well elevations (m)
well_ls_elevs_m = raster_point_value(input_raster, dec_wells_all_working)

#  add computed well elevations as column in DEC wells df
dec_wells_all_working['ls_elev_m'] = well_ls_elevs_m

#  compute groundwater elevations in meters, using DEC supplied DTW in ft and computed LSE values in meters
dec_wells_all_working['gw_elev_m'] = dec_wells_all_working['ls_elev_m'] - (dec_wells_all_working['GW_Depth'].astype(float) * 0.3048)

In [ ]:
#  add "x" and "y" columns in CRS of model for mfsetup

dec_wells_all_working['x'] = dec_wells_all_working.geometry.x
dec_wells_all_working['y'] = dec_wells_all_working.geometry.y

In [ ]:
dec_wells_all_working['group'].unique().tolist()

**Looks like in the end we clipped out all of the DEC water wells and are only using the DEC wells supplied by NYWSC**

In [ ]:
dec_wells_all_working.to_csv('../processed_data/NY_DEC_GW_sites.csv')